# RvF - Starter Code Workbook

This is a workbook that you can upload to Google Colab to work on the project. It will download all the correct files and details to work on model development and improving.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Change this to the folder containing your Kaggle API key (kaggle.json)
%env KAGGLE_KEY_FOLDER=MDST
!mkdir data
!export KAGGLE_CONFIG_DIR=/content/drive/MyDrive/$KAGGLE_KEY_FOLDER && wget -O - "https://raw.githubusercontent.com/MichiganDataScienceTeam/W24-RvF/main/data/download.sh" | bash -s rvf10k

Mounted at /content/drive
env: KAGGLE_KEY_FOLDER=MDST
--2024-03-10 18:46:48--  https://raw.githubusercontent.com/MichiganDataScienceTeam/W24-RvF/main/data/download.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2600 (2.5K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               WARNING: Your Python virtual environment should be enabled while running this script!
-                   100%[===================>]   2.54K  --.-KB/s    in 0s      

2024-03-10 18:46:48 (5.11 MB/s) - written to stdout [2600/2600]

Current Python Environment: /usr/local/bin/python
ls: cannot access 'data/rvf10k': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>


In [ ]:
!rm -r W24-RvF starter_code
!git clone -q https://github.com/MichiganDataScienceTeam/W24-RvF.git
!mv W24-RvF/starter_code .
!rm -r W24-RvF

rm: cannot remove 'W24-RvF': No such file or directory
rm: cannot remove 'starter_code': No such file or directory


**KEY**: Make sure to save your work after every meeting! We recommend you do so via GitHub, but its not necessary.

The following sample code is the basics you need to get started with model development.

In [43]:
import torch
import torchvision.transforms.v2 as v2
from starter_code.dataset import RvFDataset

mean = torch.zeros((3,))
variance = torch.zeros((3,))
tensor_converter = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
train_dataset = RvFDataset("train", "data/rvf10k")

for image, _ in train_dataset:
  mean += tensor_converter(image).mean(dim=(1, 2))
mean /= len(train_dataset)

for image, _ in train_dataset:
  image = tensor_converter(image)
  variance += ((image - mean.view(3, 1, 1))**2).mean(dim=(1, 2))
std = torch.sqrt(variance/len(train_dataset))

In [44]:
def preprocess(image) -> torch.Tensor:
  """
  Preprocesses an image by applying a series of transformation.

  Args:
      image (npt.ArrayLike): The input image to be preprocessed.

  Returns:
      torch.Tensor: The preprocessed image as a tensor.
  """
  # TODO: Edit this function to more preprocessing steps to improve model performance.
  tensor = torch.tensor(image, dtype=torch.float32)

  # Default image preprocessing in tutorial
  normalizer = v2.Normalize(mean=mean, std=std)
  cropper = v2.RandomCrop(size=(28, 28))
  blurrer = v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.))

  # Apply to image
  # TODO: implement this step
  tensor

  return tensor.permute(2,0,1)

In [45]:
from torch import nn, optim
from math import sqrt

class Model(torch.nn.Module):
    # TODO: Define your model here!

    def __init__(self):
        """Constructor for the neural network."""
        super(Model, self).__init__()        # Call superclass constructor

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(in_features=4096, out_features=10)
        self.dropout = nn.Dropout(p=0.3)

        self.init_weights()


    def init_weights(self):
        """Initialize all model parameters (weights and biases) in all layers to desired distributions"""
        ## TODO: initialize the parameters for your network
        torch.manual_seed(42)

        for conv in [self.conv1, self.conv2, self.conv3]:
            C_in = conv.weight.size(1)
            nn.init.normal_(conv.weight, 0.0, 1 / sqrt(5 * 5 * C_in))
            nn.init.constant_(conv.bias, 0.0)

        ## TODO: initialize the parameters for [self.fc_1]
        nn.init.normal_(self.fc_1.weight, mean=0, std=1 / sqrt(8 * 2 * 2))
        nn.init.constant_(self.fc_1.bias, 0.0)


    def forward(self, x):
        N, C, H, W = x.shape
        z1 = self.conv1(x)
        h1 = self.relu(z1)
        p1 = self.pool(h1)

        z2 = self.conv2(p1)
        h2 = self.relu(z2)
        p2 = self.pool(h2)

        z2 = self.conv3(p2)
        h3 = self.relu(z2)
        p3 = self.pool(h3)

        z = self.dropout(p3) # dropout layer (0.3)
        flat = self.flatten(z) # Flatten the 4D tensor to a 2D tensor for the fully connected layer
        z = self.fc_1(flat)

        return z

In [46]:
from starter_code.dataset import get_loaders
from starter_code.train import train_model, plot_performance, load_model

train_loader, val_loader = get_loaders(preprocessor=preprocess)

model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.01) # TODO: Change the optimizer to explore different options
criterion = torch.nn.CrossEntropyLoss() # TODO: Change the criterion to explore different options

history = train_model(model, criterion, optimizer, train_loader, val_loader, epochs=5)
plot_performance(history)

# Load the model from the training run
load_model(model, "checkpoints", 0) # change epoch from 0 to something else

  0%|          | 0/219 [00:02<?, ?it/s]


KeyboardInterrupt: 

**KEY**: At the end of each work session, submit this workbook via slack! This will allow the project leads to train your model on the larger 140k dataset over the week!